In [1]:
import pandas_datareader.data as web
from statsmodels.regression.linear_model import OLS
from statsmodels.tools.tools import add_constant
from pandas import DataFrame
import DatastreamDSWS as dsws
from os import environ
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import urllib
from time import sleep
import numpy as np

In [2]:
ds = dsws.Datastream(
    environ['DATASTREAM_USER'],
    environ['DATASTREAM_PASSWORD']
    )

In [3]:
date_yesterday_dash = (datetime.utcnow()- timedelta(days=1)).strftime('%Y-%m-%d')
date_yesterday = (datetime.utcnow()- timedelta(days=1)).strftime('%Y%m%d')
lm_end = datetime.utcnow().replace(day=1) - timedelta(days=1)
lm_end = lm_end.strftime('%Y%m%d')
lm_end_dash = datetime.strptime(lm_end, '%Y%m%d').strftime('%Y-%m-%d')
lm_p1_end_dash = (datetime.strptime(lm_end, '%Y%m%d') + \
    relativedelta(months=1)).strftime('%Y-%m-%d')

In [5]:
tickers = ['AAPL.O','NVDA.O','AMZN.O','ADM.N','SHOP.TO','WPM.TO','RY.TO','DOO.TO','QCOM.O','COIN.O','CIX.TO',
                   'QSR.TO','BX.N']
tickers = [f'<{ticker}>' for ticker in tickers]

In [10]:
rel_start_date = '-3Y'

market_tickers = ['MSWRLD$','U:IMTM','U:IVLU','U:EFG','U:ISZE']
factor_columns = {
        'MSWRLD$':'Mkt',
        'U:IMTM':'Momentum',
        'U:IVLU':'Value',
        'U:EFG':'Growth',
        'U:ISZE':'Size'
        }
factor_columns = {k: factor_columns[k] for k in market_tickers \
     if k in factor_columns}
final_factor_columns = factor_columns
final_factor_columns['_'] = 'Mkt-Rf'
final_factor_columns = list(final_factor_columns.values())
final_factor_columns.remove('Mkt')
factor_columns[ 'daily_rf_ret'] = 'Rf'
rf_ticker = ['TRCN3MT']


In [11]:
'''Portfolio Returns'''
field_list = ['X(RI)~C$'] # this should be a list
#"X(P)~C$"
df_port_ret = ds.get_data(
#     '<TSM.N>'
                    ','.join(tickers)
                    # , start = "-4Y"
                    # , end = ""
                          , start=rel_start_date
                          , end=date_yesterday_dash
                  , kind=1
                  , freq="W"
                  , fields=field_list
                  )

In [12]:
'''Factor Returns'''
field_list = ['X(RI)~C$'] # this should be a list
#"X(P)~C$"
df_market_ret = ds.get_data(
                    ','.join(market_tickers)
                  , start=rel_start_date
                  , end=date_yesterday_dash
                  , kind=1
                  , freq="W"
                  , fields=field_list
                  )

In [48]:
'''Risk-Free Instrument Returns'''
field_list = ['X(RI)~C$'] # this should be a list
#"X(P)~C$"
df_rf_returns = ds.get_data(
                    ','.join(rf_ticker)
                  , start=rel_start_date
                  , end=date_yesterday_dash
                  , kind=1
                  , freq="W"
                  , fields=['RY']
                  )

In [49]:
df_rf_returns.columns = df_rf_returns.columns.droplevel([1,2])
df_rf_returns['daily_rf_ret'] = df_rf_returns['TRCN3MT'].div(100).add(1).pow(1/(252/5)).sub(1)

In [51]:
df_rf_returns['daily_rf_ret'].tail()

Dates
2023-12-11    0.000974
2023-12-18    0.000974
2023-12-25    0.000978
2024-01-01    0.000978
2024-01-08    0.000978
Name: daily_rf_ret, dtype: float64

In [52]:
def get_returns_from_prices(df, cull=True):
    df_returns = np.log(df) - np.log(df.shift())
    if cull:
        df_returns.dropna(inplace=True)
    else:
        df_returns = df_returns[1:]
    df_returns.columns = df_returns.columns.droplevel([1,2])
    df_returns = df_returns.loc[~(df_returns==0).all(axis=1)] # drop rows with all 0s
    return df_returns

In [53]:
df_port_returns = get_returns_from_prices(df_port_ret, cull=False)
df_market_returns = get_returns_from_prices(df_market_ret, cull=False)

In [54]:
df_port_returns.head()

Instrument,<AAPL.O>,<NVDA.O>,<AMZN.O>,<ADM.N>,<SHOP.TO>,<WPM.TO>,<RY.TO>,<DOO.TO>,<QCOM.O>,<COIN.O>,<CIX.TO>,<QSR.TO>,<BX.N>
Dates,,,,,,,,,,,,,
2021-01-11,0.001429,0.042764,-0.018232,0.047350,0.073325,-0.060301,0.029740,0.028787,0.054733,NaN,0.084362,0.062295,-0.000176
2021-01-18,-0.017560,-0.060759,-0.006395,-0.000122,0.009743,-0.056188,0.004257,0.036483,0.003066,NaN,0.009686,-0.008586,0.007280
2021-01-25,0.115828,0.058726,0.058162,0.015738,0.012786,-0.003692,-0.008624,-0.009090,0.044315,NaN,-0.006045,-0.062439,0.024226
2021-02-01,-0.056458,-0.024019,0.021673,-0.043469,-0.042671,0.080388,-0.016793,0.000803,-0.010359,NaN,0.006045,0.008729,0.071750
2021-02-08,0.015357,0.080320,-0.012563,0.061197,0.154199,-0.048396,0.024665,0.091137,-0.098566,NaN,0.047640,0.004742,0.024329


In [55]:
df_market_returns.head()

Instrument,MSWRLD$,U:IMTM,U:IVLU,U:EFG,U:ISZE
Dates,,,,,
2021-01-11,0.027309,0.016510,0.038702,0.009542,0.022656
2021-01-18,-0.009132,-0.017330,-0.001903,-0.011725,-0.003363
2021-01-25,0.014809,0.026872,-0.004952,0.019915,0.006495
2021-02-01,-0.014515,-0.017690,-0.019695,-0.015227,-0.006054
2021-02-08,0.029337,0.014206,0.041024,0.011140,0.022110


In [57]:
df_factors = df_market_returns.join(df_rf_returns)
df_factors.drop(columns='TRCN3MT', inplace=True)
df_factors.rename(
    columns=factor_columns,
    inplace=True
    )

In [58]:
df_port_returns

Instrument,<AAPL.O>,<NVDA.O>,<AMZN.O>,<ADM.N>,<SHOP.TO>,<WPM.TO>,<RY.TO>,<DOO.TO>,<QCOM.O>,<COIN.O>,<CIX.TO>,<QSR.TO>,<BX.N>
Dates,,,,,,,,,,,,,
2021-01-11,0.001429,0.042764,-0.018232,0.047350,0.073325,-0.060301,0.029740,0.028787,0.054733,NaN,0.084362,0.062295,-0.000176
2021-01-18,-0.017560,-0.060759,-0.006395,-0.000122,0.009743,-0.056188,0.004257,0.036483,0.003066,NaN,0.009686,-0.008586,0.007280
2021-01-25,0.115828,0.058726,0.058162,0.015738,0.012786,-0.003692,-0.008624,-0.009090,0.044315,NaN,-0.006045,-0.062439,0.024226
2021-02-01,-0.056458,-0.024019,0.021673,-0.043469,-0.042671,0.080388,-0.016793,0.000803,-0.010359,NaN,0.006045,0.008729,0.071750
2021-02-08,0.015357,0.080320,-0.012563,0.061197,0.154199,-0.048396,0.024665,0.091137,-0.098566,NaN,0.047640,0.004742,0.024329
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-11,0.021626,0.026356,0.009246,0.008035,0.006901,-0.021680,0.017852,-0.016879,0.048263,-0.019964,0.000000,-0.010358,-0.020292
2023-12-18,0.001454,0.058906,0.042077,-0.049654,0.043730,0.009523,0.053761,0.072883,0.027492,0.093348,0.053737,0.028036,0.096193
2023-12-25,-0.021480,-0.034938,-0.013948,-0.021960,-0.013940,0.011098,0.009546,0.082389,0.003185,0.124627,-0.022098,0.024708,0.027138


In [59]:
df_port_returns.sub(df_factors['Rf'], axis='index')

Instrument,<AAPL.O>,<NVDA.O>,<AMZN.O>,<ADM.N>,<SHOP.TO>,<WPM.TO>,<RY.TO>,<DOO.TO>,<QCOM.O>,<COIN.O>,<CIX.TO>,<QSR.TO>,<BX.N>
Dates,,,,,,,,,,,,,
2021-01-11,0.001415,0.042750,-0.018245,0.047336,0.073311,-0.060315,0.029727,0.028773,0.054719,NaN,0.084348,0.062281,-0.000190
2021-01-18,-0.017571,-0.060769,-0.006406,-0.000132,0.009732,-0.056199,0.004246,0.036472,0.003056,NaN,0.009675,-0.008596,0.007269
2021-01-25,0.115814,0.058712,0.058148,0.015724,0.012772,-0.003706,-0.008638,-0.009104,0.044301,NaN,-0.006059,-0.062453,0.024212
2021-02-01,-0.056471,-0.024032,0.021660,-0.043482,-0.042684,0.080375,-0.016806,0.000790,-0.010372,NaN,0.006032,0.008716,0.071737
2021-02-08,0.015343,0.080305,-0.012577,0.061182,0.154184,-0.048410,0.024650,0.091122,-0.098581,NaN,0.047625,0.004727,0.024314
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-11,0.020652,0.025382,0.008272,0.007060,0.005927,-0.022655,0.016878,-0.017853,0.047289,-0.020938,-0.000974,-0.011332,-0.021266
2023-12-18,0.000480,0.057931,0.041103,-0.050628,0.042756,0.008549,0.052787,0.071909,0.026518,0.092373,0.052763,0.027062,0.095219
2023-12-25,-0.022458,-0.035916,-0.014926,-0.022938,-0.014918,0.010120,0.008568,0.081411,0.002207,0.123649,-0.023076,0.023730,0.026161


In [60]:
df_factors

Instrument,Mkt,Momentum,Value,Growth,Size,Rf
Dates,,,,,,
2021-01-11,0.027309,0.016510,0.038702,0.009542,0.022656,0.000014
2021-01-18,-0.009132,-0.017330,-0.001903,-0.011725,-0.003363,0.000011
2021-01-25,0.014809,0.026872,-0.004952,0.019915,0.006495,0.000014
2021-02-01,-0.014515,-0.017690,-0.019695,-0.015227,-0.006054,0.000013
2021-02-08,0.029337,0.014206,0.041024,0.011140,0.022110,0.000015
...,...,...,...,...,...,...
2023-12-11,0.012057,0.008568,0.009274,0.011862,0.012878,0.000974
2023-12-18,0.012041,-0.009791,-0.005642,0.009068,0.007210,0.000974
2023-12-25,-0.002811,0.006756,0.011415,0.004198,0.008915,0.000978


In [61]:
ff_portfolio_data = df_port_returns.sub(df_factors['Rf'], axis='index').join(df_factors).dropna()
ff_portfolio_data['Mkt-Rf'] = ff_portfolio_data['Mkt'].sub(ff_portfolio_data['Rf'])
ff_factor_data = ff_portfolio_data[final_factor_columns] 
# ['Mkt-Rf','Momentum','Value','Growth','Size']
ff_portfolio_data = ff_portfolio_data[df_port_returns.columns]

In [62]:
ff_portfolio_data.head()

Instrument,<AAPL.O>,<NVDA.O>,<AMZN.O>,<ADM.N>,<SHOP.TO>,<WPM.TO>,<RY.TO>,<DOO.TO>,<QCOM.O>,<COIN.O>,<CIX.TO>,<QSR.TO>,<BX.N>
Dates,,,,,,,,,,,,,
2021-04-26,-0.010693,-0.002263,0.001108,0.001768,0.006436,-0.015148,-0.000979,0.041939,0.017259,-0.099164,0.015892,-0.009923,0.083910
2021-05-03,-0.026081,-0.052080,-0.016392,0.064901,-0.047286,0.008897,0.011391,-0.043133,-0.020838,-0.044223,0.037947,0.050727,0.015286
2021-05-10,-0.056803,-0.053858,-0.074232,0.030761,-0.053591,-0.005513,0.019187,-0.014478,-0.078380,-0.017263,0.025628,-0.034475,-0.013144
2021-05-17,-0.005779,-0.008248,0.023539,-0.001049,0.014699,0.086012,0.015724,-0.130125,0.005452,-0.168503,0.036111,-0.025227,-0.010602
2021-05-24,0.006001,0.096680,-0.008348,-0.010240,0.108212,-0.004249,0.009585,-0.000328,0.023127,-0.097465,0.018698,0.008899,0.037888


In [63]:
ff_factor_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 142 entries, 2021-04-26 to 2024-01-08
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Momentum  142 non-null    float64
 1   Value     142 non-null    float64
 2   Growth    142 non-null    float64
 3   Size      142 non-null    float64
 4   Mkt-Rf    142 non-null    float64
dtypes: float64(5)
memory usage: 6.7+ KB


In [64]:
betas = []
for industry in ff_portfolio_data:
    step1 = OLS(endog=ff_portfolio_data.loc[
        ff_factor_data.index, industry
        ],
                exog=add_constant(ff_factor_data)).fit()
    betas.append(step1.params.drop('const'))
betas = DataFrame(betas,
                     columns=ff_factor_data.columns,
                     index=ff_portfolio_data.columns)
betas.info()


<class 'pandas.core.frame.DataFrame'>
Index: 13 entries, <AAPL.O> to <BX.N>
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Momentum  13 non-null     float64
 1   Value     13 non-null     float64
 2   Growth    13 non-null     float64
 3   Size      13 non-null     float64
 4   Mkt-Rf    13 non-null     float64
dtypes: float64(5)
memory usage: 1.2+ KB


In [65]:
betas

Instrument,Momentum,Value,Growth,Size,Mkt-Rf
Instrument,,,,,
<AAPL.O>,-0.373190,-0.227600,0.116247,-0.777966,2.372166
<NVDA.O>,0.294463,-0.202465,2.063874,-2.497104,2.474276
<AMZN.O>,-0.238180,-0.007491,0.908407,-1.124421,1.982295
<ADM.N>,1.035772,-0.241881,-1.913797,0.741981,1.031203
<SHOP.TO>,-0.144244,-1.350321,1.366260,0.174778,2.639793
<WPM.TO>,0.273740,-1.769413,-1.199863,3.804370,-0.537397
<RY.TO>,-0.176641,0.169484,-1.138983,1.508699,0.606149
<DOO.TO>,0.356287,0.225651,0.017791,0.116576,1.181420
<QCOM.O>,0.037960,-0.574914,0.378337,-0.103435,1.622872


In [66]:
betas.mean()

Instrument
Momentum   -0.074001
Value      -0.418095
Growth     -0.034002
Size        0.400459
Mkt-Rf      1.705484
dtype: float64